# Pair Programming Limpieza V

En este caso trabajaremos con el dataframe que limpiamos en el ejercicio de pair de Limpieza III donde limpiabamos las columnas de species, age, etc. y eliminabamos los outliers. Hoy volveremos a gestionar valores nulos, pero en este caso usaremos los métodos de imputación de sklearn.

Es el momento de ponernos a trabajar con los valores nulos de nuevo 💪🏽. A lo largo de este ejercicio de pair programming vamos a intentar eliminar los valores nulos de nuestras columnas. En la lección hemos aprendido varios métodos de skelarn intentemos aplicarlos todos. Manos a la obra!

In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import sidetable
import warnings
warnings.filterwarnings('ignore')

In [8]:
from sklearn.impute import SimpleImputer

In [23]:
from sklearn.impute import KNNImputer

In [5]:
df = pd.read_csv("datos/3.ataques_limpio.csv", index_col = 0)
df.head(2)

,case_number,year,type,country,species_,date,mes,fatal,sexo,especie_cat,age_limpio
0,2018.06.25,2018,Boating,usa,White shark,25-Jun-2018,Jun,n,F,White,57.0
1,2018.06.03.a,2018,Unprovoked,brazil,Tiger shark,03-Jun-2018,Jun,y,M,Tiger,18.0


In [6]:
df_cat=df.select_dtypes(include="object")
df_cat.stb.missing()

,missing,total,percent
species_,126,1672,7.535885
especie_cat,126,1672,7.535885
fatal,97,1672,5.801435
mes,50,1672,2.990431
sexo,14,1672,0.837321
country,10,1672,0.598086
case_number,0,1672,0.000000
type,0,1672,0.000000
date,0,1672,0.000000


In [7]:
df_num=df.select_dtypes(include=np.number)
df_num.stb.missing()

,missing,total,percent
age_limpio,158,1672,9.449761
year,0,1672,0.000000


### 1. Es el momento de eliminar los nulos:

1.1 Reemplazad los valores nulos de la columna age por la media de la edad usando el método SimpleImputer.

In [9]:
imputer = SimpleImputer(strategy="mean", missing_values= np.nan)

In [10]:
imputer = imputer.fit(df[["age_limpio"]])

In [11]:
df["age"] = imputer.transform(df[["age_limpio"]])

In [12]:
df["age"].isnull().sum()

0

In [13]:
df["age_limpio"].isnull().sum()

158

1.2 Reemplazad los valores nulos de la columna sex por la moda, usando el método SimpleImputer.
💡 Pista 💡 La moda en este tipo de aproximación se indica como most_frequent.

In [15]:
df["sexo"].isnull().sum()

14

Hay 14 nulos

In [17]:
imputer2 = SimpleImputer(strategy="most_frequent", missing_values= np.nan)

In [18]:
imputer2 = imputer2.fit(df[["sexo"]])

In [19]:
df["sexo"] = imputer2.transform(df[["sexo"]])

In [20]:
df["sexo"].isnull().sum()

0

1.3 Reemplazad los valores nulos de la columna type por el valor más frecuente (la moda) con el método SimpleImputer.

In [22]:
# No existen valores nulos
df["type"].isnull().sum()

0

In [21]:
# Esta sería la aplicación si existieran nulos
imputer3 = SimpleImputer(strategy="most_frequent", missing_values= np.nan)
imputer3 = imputer3.fit(df[["type"]])
df["type"] = imputer2.transform(df[["type"]])
df["type"].isnull().sum()

0

1.4 Utilizad el método KNN Imputer para reemplazar todos los valores nulos de las columnas numéricas.

In [24]:
df_num.columns

Index(['year', 'age_limpio'], dtype='object')

In [26]:
imputerKNN = KNNImputer(n_neighbors=6)
imputerKNN.fit(df_num)

KNNImputer(n_neighbors=6)

In [27]:
cat_numKNN = imputerKNN.transform(df_num)

In [28]:
df_KNN = pd.DataFrame(cat_numKNN, columns=df_num.columns)
df_KNN.head(2)

,year,age_limpio
0,2018.0,57.0
1,2018.0,18.0


1.5 Utilizad el método Iterative Imputer para reemplazar todos los valores nulos de las columnas numéricas.

1.6 ¿Podríais explicar qué diferencia hay entre estos dos últimos métodos?

### 2. Guardad el csv para seguir trabajando con el en los siguientes ejercicios de pair